## Tile Encoding

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 

import os 
os.environ["USE_TRANSPATH"] = "True"
print(os.environ["USE_TRANSPATH"])

import torch
import pandas as pd
from pkg.wsi_mil.tile_wsi.encoders import CTranspathModel
from scripts.encode_tiles import encode_tiles
from pkg.wsi_mil.tile_wsi.pca_partial import pca_savings


device = "mps"
gt_filepath = "/Users/aliceblondel/Desktop/WSI_vesper/data/Selected_Contour_annotation.xlsx"
data_folder = "/Users/aliceblondel/Desktop/WSI_vesper/data/"
embedding_folder = "/Users/aliceblondel/Desktop/WSI_vesper/data/Embeddings_complete/ctranspath/"
save_tiles_img = True

print(f"Device: {device}")
print(f"GT file path: {gt_filepath}")
print(f"Data folder: {data_folder}")
print(f"Embedding folder: {embedding_folder}")

df = pd.read_excel(gt_filepath, index_col=0)
model = CTranspathModel().to(device)

model.eval()
with torch.no_grad():
    for zone_id in df["zone_id"].unique():
        print(zone_id)
        tile_embeddings, tile_xys, save_folder = encode_tiles(
            model, 
            zone_id,
            df,
            data_folder,
            embedding_folder,
            magnification_tile=10, 
            max_tiles_per_slide=None,
            device=device, 
            mask_tolerance=0.9,
            save_tiles_img=save_tiles_img,
        )

pca_path = save_folder / "pca" 
pca_path.mkdir(exist_ok=True)
pca_savings(model.pca, pca_path)

## Old version

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import os 
os.environ["USE_TRANSPATH"] = "True"
print(os.environ["USE_TRANSPATH"])

from pkg.wsi_mil.tile_wsi.tiler import ImageTiler
from pkg.wsi_mil.tile_wsi.pca_partial import main as incremental_pca
from pkg.wsi_mil.tile_wsi.arguments import get_arguments
import os
import numpy as np
from glob import glob

In [ ]:
params = {
    "path_wsi" :  "/Users/aliceblondel/Desktop/WSI_vesper/data/Slides/",
    "size" :  224,
    "tiler" :  "imagenet",
    "level" :  2,
    "path_outputs" :  "/Users/aliceblondel/Desktop/WSI_vesper/data/Embeddings_complete/",
}

args = get_arguments(params)
args.path_wsi

In [ ]:
outpath = os.path.join(args.path_outputs, args.tiler, f'level_{args.level}')
if os.path.isfile(args.path_wsi):
    it = ImageTiler(args=args)
    it.tile_image()
else:
    dirs = []
    extensions = set(['.ndpi', '.svs', '.tif'])
    all_files = glob(os.path.join(args.path_wsi, '*.*'))
    files = [f for f in all_files if os.path.splitext(f)[1] in extensions]
    assert len(files) > 0, f"No files with extension {extensions} in {args.path_wsi}"
    # for f in files:
    f = '/Users/aliceblondel/Desktop/WSI_vesper/data/Slides/VB002 A-HES - 2022-01-27 10.35.08.ndpi'
    args.path_wsi = f
    it = ImageTiler(args=args)
    it.tile_image()

#PCA
outpath = '/Users/aliceblondel/Desktop/WSI_vesper/data/Embeddings_complete/ctranspath/level_2'
if args.tiler != 'simple':
    os.chdir(outpath)
    os.makedirs('pca', exist_ok=True)
    raw_args = ['--path', '.']
    ipca = incremental_pca(raw_args)
    os.makedirs(os.path.join('./mat_pca'), exist_ok=True)
    files = glob('./mat/*.npy')
    for f in files:
        m = np.load(f)
        mp = ipca.transform(m)
        np.save(os.path.join('mat_pca', os.path.basename(f)), mp)
